In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

2024-02-29 01:58:01.740218: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 01:58:01.787178: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-29 01:58:02.004465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 01:58:02.004518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 01:58:02.044268: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# Define paths to images and masks folders
images_dir = "../Dataset/train/"
masks_dir = "../Masks/"

In [ ]:
images, masks = load_data(images_dir, masks_dir)

# Print shapes of images and masks to check if they are loaded correctly
print("Number of images:", len(images))
print("Number of masks:", len(masks))

In [4]:
def load_data(images_dir, masks_dir):
    images = []
    masks = []
    class_labels = {"root": 0, "seed": 1, "shoot": 2, "occluded_root": 3}
    
    mask_files = os.listdir(masks_dir)
    
    for img_file in os.listdir(images_dir):
        if img_file.endswith(".png"):
            img_path = os.path.join(images_dir, img_file)
            corresponding_mask_file = None
            
            for mask_file in mask_files:
                if mask_file.startswith(img_file[:-4]):
                    corresponding_mask_file = mask_file
                    break
            
            if corresponding_mask_file:
                mask_path = os.path.join(masks_dir, corresponding_mask_file)
                class_name = corresponding_mask_file.split("_")[-2]  # Extract class name from filename
                class_label = class_labels.get(class_name, -1)  # Get class label
                
                if class_label != -1:
                    mask = np.array(load_img(mask_path, color_mode="grayscale"))
                    mask = (mask > 0).astype(np.uint8)  # Convert mask to binary
                    masks.append(mask)
                    
                    img = img_to_array(load_img(img_path)) / 255.0  # Normalize image
                    images.append(img)
    
    if not images or not masks:
        raise ValueError("No images or masks found in the specified directories.")
    
    return np.array(images), np.array(masks)

In [ ]:
# Load images and masks
images, masks = load_data(images_dir, masks_dir)

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, masks, test_size=0.2, random_state=42)

In [ ]:
# Define model architecture (U-Net)
def unet_model(input_shape=(None, None, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Decoder
    up1 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv2)
    concat1 = layers.concatenate([conv1, up1], axis=3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat1)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv3)
    
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)
    
    return tf.keras.Model(inputs=inputs, outputs=outputs)

# Create U-Net model
model = unet_model()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.2)

# Plot learning curves
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model on test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Example predictions
n_examples = 5
idx = np.random.randint(0, len(X_test), n_examples)
preds = model.predict(X_test[idx])

# Plot example predictions
plt.figure(figsize=(15, 10))
for i in range(n_examples):
    plt.subplot(2, n_examples, i + 1)
    plt.imshow(X_test[idx[i]])
    plt.title('Input Image')
    plt.axis('off')
    
    plt.subplot(2, n_examples, i + n_examples + 1)
    plt.imshow(preds[i].squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')
plt.show()